In [ ]:
import sys
sys.path.append('/SEE-MTDA/see')
from sc.mesher.mesher import Mesher
import numpy as np
import os
import glob
from pathlib import Path
from sc.datasets.shared_utils import *
import open3d as o3d
from easydict import EasyDict
import yaml
from PIL import Image

def cfg_from_yaml_file(cfg_file):
    cfg = EasyDict()
    print("\n----- Cfgs -----")
    with open(cfg_file, 'r') as f:
        yaml_config = yaml.safe_load(f)
        print(yaml.dump(yaml_config))

    cfg.update(EasyDict(yaml_config))

    return cfg

cfg_file = '/SEE-MTDA/see/sc/cfgs/NUS-GM-ORH005.yaml'
cfg = cfg_from_yaml_file(cfg_file)
mesher = Mesher(cfg=cfg, cfg_path=cfg_file)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

----- Cfgs -----
DATASET:
  CLASSES:
  - car
  CUSTOM: true
  DET2D_MASK: false
  LIDAR_NSWEEPS: 2
  NAME: nuscenes
DATA_PATH: /SEE-MTDA/data/nuscenes/custom_t4025-v3980
MESHER:
  MESH_ALGORITHM:
    LOWER_RADIUS: 0.01
    NAME: ball_pivoting
    UPPER_RADIUS: 1.155
  MIN_LIDAR_PTS_TO_MESH: 50
  NAME: gt_mesh
  REPLACE_OBJECT_WITH_MESH:
    POINT_DISTANCE_THRESH: 0.1
  SAMPLING:
    NAME: vres_ring_based_sampling
    OPTIMAL_RING_HEIGHT: 0.05
    TYPE: poisson
  VRES: 1.25

Loading NuScenes tables for version v1.0-trainval...


# Export points in GT box

This is to assess shape completion with perfect data

In [ ]:
mesher.data_obj.__len__()

In [2]:
mesher.min_lidar_pts_to_mesh = 5

for sample_idx in range(mesher.data_obj.__len__()):
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx)

    for object_id in range(len(pcd_gtboxes['gt_boxes'])):    

        object_pcd = pcd_gtboxes['pcd'].crop(pcd_gtboxes['gt_boxes'][object_id])

        if len(object_pcd.points) < mesher.min_lidar_pts_to_mesh:
            continue

        box_pts = np.asarray(pcd_gtboxes['gt_boxes'][object_id].get_box_points())

        if mesher.data_obj.dataset_name == 'nuscenes':
            fname = f'nsweeps-{mesher.data_obj.nsweeps}_frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'
        else:
            fname = f'frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'

        car_dir = Path(f'{mesher.data_obj.root_dir}/exported/cars')
        car_dir.mkdir(parents=True, exist_ok=True)
        o3d.io.write_point_cloud(str(car_dir / f'{fname}.pcd'), object_pcd)

        box_dir = Path(f'{mesher.data_obj.root_dir}/exported/bboxes')    
        box_dir.mkdir(parents=True, exist_ok=True)
        np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')

        print(f"frame-{sample_idx}: Exported {object_id}/{len(pcd_gtboxes['gt_boxes'])} | car_pts-{len(object_pcd.points)}")


frame-1: Exported 0/1 | car_pts-9
frame-2: Exported 0/1 | car_pts-53


KeyboardInterrupt: 

# Export points in mask/bbox

This is to test out lidar segmentation on the points within a box

In [4]:
def export_lidarseg(mesher, export_tag, camera_channel, append_labels=False, add_ground_lift=False):
    if export_tag == 'bbox':
        use_bbox = True
    else:
        use_bbox = False
    
    instance_dataset_id = 0
    for sample_idx in range(mesher.data_obj.__len__()):
        
        print(f'Frame #{sample_idx}/{mesher.data_obj.__len__()}')
        i_clouds = mesher.data_obj.get_mask_instance_clouds(sample_idx, use_bbox=use_bbox, camera_channels=camera_channel)
        pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx, add_ground_lift=add_ground_lift)    
        inst_labels = mesher.data_obj.get_camera_instances(sample_idx, channel=camera_channel)
        
        for object_id, i_cloud in enumerate(i_clouds):
            mask_pcd = convert_to_o3dpcd(i_clouds[object_id])
            cropped_pcds, num_pts_in_gt_for_mask = [], []        

            if len(i_cloud) < 5:
                continue

            for i in range(len(pcd_gtboxes['gt_boxes'])):    
                cropped = mask_pcd.crop(pcd_gtboxes['gt_boxes'][i])
                num_pts_in_gt_for_mask.append(len(cropped.points))
                cropped_pcds.append(cropped)            

            if not num_pts_in_gt_for_mask or max(num_pts_in_gt_for_mask) == 0:
                labelled_pts = np.hstack([i_cloud[:,:3], np.zeros((len(i_cloud),1))])
                box_pts = np.zeros((8,3))
                num_points_in_gt = 0
            else:
                gtbox_idx = np.argmax(np.array(num_pts_in_gt_for_mask)) 

                cropped_np = np.array(cropped_pcds[gtbox_idx].points)
                obj_pts_labelled = np.hstack([cropped_np, 10*np.ones((len(cropped_np),1))])

                dist = mask_pcd.compute_point_cloud_distance(cropped_pcds[gtbox_idx])
                cropped_inds = np.where(np.asarray(dist) < 0.01)[0]
                mask_without_obj = np.asarray(mask_pcd.select_by_index(cropped_inds, invert=True).points)
                mask_pts_labelled = np.hstack([mask_without_obj, np.zeros((len(mask_without_obj),1))])

                labelled_pts = np.vstack([mask_pts_labelled, obj_pts_labelled])
                box_pts = np.asarray(pcd_gtboxes['gt_boxes'][gtbox_idx].get_box_points())
                num_points_in_gt = len(cropped_np)

            # Save pcl
            fname = f"{instance_dataset_id:06}"
            car_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/{camera_channel}/pointcloud')
            car_dir.mkdir(parents=True, exist_ok=True)
            car_fname = str(car_dir / f'{fname}.bin')
            points = np.float32(labelled_pts)
            points.tofile(car_fname)

            # Save points labels
            label_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/{camera_channel}/labels')
            label_dir.mkdir(parents=True, exist_ok=True)
            label_fname = str(label_dir / f'{fname}.label')
            labels = np.uint32(labelled_pts[:,3])
            labels.tofile(label_fname)

            # Save gt bboxes
            box_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/{camera_channel}/gt_bboxes')    
            box_dir.mkdir(parents=True, exist_ok=True)
            np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')

            # Save img
            img_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/{camera_channel}/image')    
            img_dir.mkdir(parents=True, exist_ok=True)
            img = mesher.data_obj.get_image(sample_idx, channel=camera_channel)
            imglabels = inst_labels[object_id]
            ann_img = draw_lidar_on_image([], img, instances=[imglabels], display=False)
            Image.fromarray(ann_img).save(str(img_dir / f'{fname}.png'))
            
            sample_record = mesher.data_obj.sample_records[0]
            frame_token = sample_record['token']
            pc_path = mesher.data_obj.nusc.get_sample_data_path(sample_record['data']['LIDAR_TOP'])
            img_path = mesher.data_obj.nusc.get_sample_data_path(sample_record['data'][camera_channel])
            origpath_dir = Path(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_{export_tag}/{camera_channel}/fpath')
            origpath_dir.mkdir(parents=True, exist_ok=True)
            with open(str(origpath_dir / f'{fname}.txt'), 'w') as f:
                f.write(f'sample_idx,{sample_idx:06}\n')
                f.write(f'frame_token,{frame_token}\n')
                f.write(f'pc_path,{pc_path}\n')
                f.write(f'img_path,{img_path}\n')
            
            print(f"car #{instance_dataset_id}: Exported {object_id+1}/{len(i_clouds)} | total pts-{len(labelled_pts)}, gt pts - {num_points_in_gt}")
            instance_dataset_id = instance_dataset_id + 1

In [ ]:
mesher.data_obj.camera_channels = ['CAM_FRONT']
for camera in mesher.data_obj.camera_channels:
    print(f"Camera: {camera}")
    export_lidarseg(mesher, export_tag='bbox', camera_channel=camera, add_ground_lift=True)

Camera: CAM_FRONT
Frame #0/4025
car #0: Exported 1/12 | total pts-44, gt pts - 10
car #1: Exported 2/12 | total pts-34, gt pts - 23
car #2: Exported 3/12 | total pts-10, gt pts - 9
car #3: Exported 4/12 | total pts-5, gt pts - 2
car #4: Exported 5/12 | total pts-24, gt pts - 10
car #5: Exported 6/12 | total pts-15, gt pts - 5
car #6: Exported 7/12 | total pts-17, gt pts - 3
car #7: Exported 8/12 | total pts-6, gt pts - 6
car #8: Exported 9/12 | total pts-9, gt pts - 3
car #9: Exported 11/12 | total pts-12, gt pts - 3
Frame #1/4025
car #10: Exported 1/10 | total pts-121, gt pts - 28
car #11: Exported 2/10 | total pts-112, gt pts - 44
car #12: Exported 3/10 | total pts-19, gt pts - 16
car #13: Exported 4/10 | total pts-42, gt pts - 13
car #14: Exported 5/10 | total pts-5, gt pts - 5
car #15: Exported 6/10 | total pts-28, gt pts - 4
car #16: Exported 7/10 | total pts-39, gt pts - 10
car #17: Exported 9/10 | total pts-12, gt pts - 6
car #18: Exported 10/10 | total pts-25, gt pts - 4
Frame 

car #156: Exported 1/7 | total pts-229, gt pts - 104
car #157: Exported 2/7 | total pts-97, gt pts - 32
car #158: Exported 3/7 | total pts-158, gt pts - 55
car #159: Exported 4/7 | total pts-119, gt pts - 30
car #160: Exported 5/7 | total pts-32, gt pts - 5
car #161: Exported 6/7 | total pts-39, gt pts - 12
Frame #17/4025
car #162: Exported 1/7 | total pts-163, gt pts - 110
car #163: Exported 2/7 | total pts-181, gt pts - 40
car #164: Exported 3/7 | total pts-51, gt pts - 4
car #165: Exported 4/7 | total pts-61, gt pts - 21
car #166: Exported 5/7 | total pts-47, gt pts - 11
car #167: Exported 6/7 | total pts-7, gt pts - 5
Frame #18/4025
car #168: Exported 1/10 | total pts-217, gt pts - 156
car #169: Exported 2/10 | total pts-287, gt pts - 101
car #170: Exported 3/10 | total pts-9, gt pts - 1
car #171: Exported 4/10 | total pts-61, gt pts - 33
car #172: Exported 5/10 | total pts-70, gt pts - 41
car #173: Exported 7/10 | total pts-15, gt pts - 0
car #174: Exported 8/10 | total pts-24, gt

car #309: Exported 1/5 | total pts-706, gt pts - 336
car #310: Exported 2/5 | total pts-11, gt pts - 0
car #311: Exported 3/5 | total pts-64, gt pts - 3
car #312: Exported 4/5 | total pts-27, gt pts - 4
car #313: Exported 5/5 | total pts-20, gt pts - 16
Frame #52/4025
car #314: Exported 1/7 | total pts-543, gt pts - 313
car #315: Exported 2/7 | total pts-91, gt pts - 2
car #316: Exported 3/7 | total pts-31, gt pts - 10
car #317: Exported 4/7 | total pts-6, gt pts - 2
car #318: Exported 5/7 | total pts-86, gt pts - 21
car #319: Exported 6/7 | total pts-14, gt pts - 9
car #320: Exported 7/7 | total pts-5, gt pts - 3
Frame #53/4025
car #321: Exported 1/7 | total pts-184, gt pts - 97
car #322: Exported 2/7 | total pts-71, gt pts - 47
car #323: Exported 3/7 | total pts-116, gt pts - 62
car #324: Exported 4/7 | total pts-39, gt pts - 2
Frame #54/4025
car #325: Exported 1/6 | total pts-384, gt pts - 171
car #326: Exported 2/6 | total pts-136, gt pts - 52
car #327: Exported 3/6 | total pts-94,

car #462: Exported 3/3 | total pts-20, gt pts - 10
Frame #84/4025
car #463: Exported 1/3 | total pts-104, gt pts - 45
car #464: Exported 2/3 | total pts-94, gt pts - 68
car #465: Exported 3/3 | total pts-80, gt pts - 42
Frame #85/4025
car #466: Exported 1/4 | total pts-228, gt pts - 171
car #467: Exported 2/4 | total pts-193, gt pts - 103
car #468: Exported 3/4 | total pts-76, gt pts - 61
car #469: Exported 4/4 | total pts-59, gt pts - 33
Frame #86/4025
car #470: Exported 1/3 | total pts-449, gt pts - 224
car #471: Exported 2/3 | total pts-43, gt pts - 32
car #472: Exported 3/3 | total pts-48, gt pts - 22
Frame #87/4025
car #473: Exported 1/3 | total pts-583, gt pts - 339
car #474: Exported 2/3 | total pts-27, gt pts - 17
car #475: Exported 3/3 | total pts-33, gt pts - 18
Frame #88/4025
car #476: Exported 1/2 | total pts-35, gt pts - 22
car #477: Exported 2/2 | total pts-24, gt pts - 12
Frame #89/4025
car #478: Exported 1/2 | total pts-45, gt pts - 27
car #479: Exported 2/2 | total pts

car #611: Exported 1/10 | total pts-65, gt pts - 61
car #612: Exported 2/10 | total pts-51, gt pts - 23
car #613: Exported 3/10 | total pts-30, gt pts - 12
car #614: Exported 4/10 | total pts-20, gt pts - 16
car #615: Exported 5/10 | total pts-7, gt pts - 4
car #616: Exported 6/10 | total pts-7, gt pts - 4
car #617: Exported 7/10 | total pts-10, gt pts - 5
car #618: Exported 9/10 | total pts-9, gt pts - 0
Frame #125/4025
car #619: Exported 1/12 | total pts-98, gt pts - 40
car #620: Exported 2/12 | total pts-40, gt pts - 13
car #621: Exported 3/12 | total pts-33, gt pts - 23
car #622: Exported 4/12 | total pts-12, gt pts - 5
car #623: Exported 5/12 | total pts-9, gt pts - 7
car #624: Exported 7/12 | total pts-21, gt pts - 16
car #625: Exported 8/12 | total pts-11, gt pts - 0
Frame #126/4025
car #626: Exported 1/13 | total pts-123, gt pts - 91
car #627: Exported 2/13 | total pts-46, gt pts - 29
car #628: Exported 3/13 | total pts-7, gt pts - 0
car #629: Exported 4/13 | total pts-16, gt p

...

car #4367: Exported 2/7 | total pts-54, gt pts - 41
car #4368: Exported 3/7 | total pts-52, gt pts - 18
car #4369: Exported 4/7 | total pts-12, gt pts - 6
Frame #732/4025
car #4370: Exported 1/6 | total pts-93, gt pts - 20
car #4371: Exported 2/6 | total pts-17, gt pts - 7
car #4372: Exported 3/6 | total pts-6, gt pts - 0
Frame #733/4025
car #4373: Exported 1/5 | total pts-45, gt pts - 16
car #4374: Exported 2/5 | total pts-51, gt pts - 1
car #4375: Exported 3/5 | total pts-28, gt pts - 7
car #4376: Exported 5/5 | total pts-10, gt pts - 0
Frame #734/4025
car #4377: Exported 1/5 | total pts-56, gt pts - 8
car #4378: Exported 2/5 | total pts-47, gt pts - 30
car #4379: Exported 3/5 | total pts-21, gt pts - 1
car #4380: Exported 4/5 | total pts-8, gt pts - 6
Frame #735/4025
car #4381: Exported 1/6 | total pts-23, gt pts - 10
car #4382: Exported 2/6 | total pts-25, gt pts - 3
car #4383: Exported 3/6 | total pts-12, gt pts - 10
car #4384: Exported 4/6 | total pts-26, gt pts - 19
Frame #736/4

car #4516: Exported 2/7 | total pts-10, gt pts - 8
car #4517: Exported 3/7 | total pts-10, gt pts - 0
Frame #768/4025
car #4518: Exported 1/8 | total pts-54, gt pts - 7
car #4519: Exported 2/8 | total pts-13, gt pts - 7
car #4520: Exported 3/8 | total pts-22, gt pts - 8
car #4521: Exported 5/8 | total pts-9, gt pts - 0
car #4522: Exported 7/8 | total pts-11, gt pts - 0
car #4523: Exported 8/8 | total pts-8, gt pts - 3
Frame #769/4025
car #4524: Exported 1/7 | total pts-56, gt pts - 11
car #4525: Exported 2/7 | total pts-24, gt pts - 6
car #4526: Exported 3/7 | total pts-6, gt pts - 4
car #4527: Exported 4/7 | total pts-7, gt pts - 0
car #4528: Exported 5/7 | total pts-6, gt pts - 0
car #4529: Exported 6/7 | total pts-9, gt pts - 0
Frame #770/4025
car #4530: Exported 1/9 | total pts-57, gt pts - 6
car #4531: Exported 2/9 | total pts-12, gt pts - 7
car #4532: Exported 3/9 | total pts-20, gt pts - 6
car #4533: Exported 4/9 | total pts-5, gt pts - 0
car #4534: Exported 5/9 | total pts-22, 

car #4669: Exported 1/4 | total pts-612, gt pts - 352
car #4670: Exported 2/4 | total pts-134, gt pts - 81
car #4671: Exported 3/4 | total pts-42, gt pts - 0
Frame #795/4025
car #4672: Exported 1/5 | total pts-697, gt pts - 413
car #4673: Exported 2/5 | total pts-212, gt pts - 99
car #4674: Exported 3/5 | total pts-48, gt pts - 1
car #4675: Exported 4/5 | total pts-8, gt pts - 1
Frame #796/4025
car #4676: Exported 1/6 | total pts-623, gt pts - 355
car #4677: Exported 2/6 | total pts-255, gt pts - 127
car #4678: Exported 3/6 | total pts-49, gt pts - 0
Frame #797/4025
car #4679: Exported 1/6 | total pts-248, gt pts - 185
car #4680: Exported 2/6 | total pts-325, gt pts - 171
car #4681: Exported 3/6 | total pts-59, gt pts - 20
car #4682: Exported 4/6 | total pts-18, gt pts - 0
car #4683: Exported 5/6 | total pts-7, gt pts - 0
Frame #798/4025
car #4684: Exported 1/5 | total pts-361, gt pts - 218
car #4685: Exported 2/5 | total pts-67, gt pts - 19
car #4686: Exported 3/5 | total pts-18, gt p

car #4817: Exported 1/4 | total pts-448, gt pts - 293
car #4818: Exported 2/4 | total pts-174, gt pts - 0
car #4819: Exported 3/4 | total pts-10, gt pts - 0
car #4820: Exported 4/4 | total pts-7, gt pts - 7
Frame #829/4025
car #4821: Exported 1/4 | total pts-690, gt pts - 405
car #4822: Exported 2/4 | total pts-88, gt pts - 41
car #4823: Exported 3/4 | total pts-72, gt pts - 20
car #4824: Exported 4/4 | total pts-14, gt pts - 0
Frame #830/4025
car #4825: Exported 1/5 | total pts-1356, gt pts - 650
car #4826: Exported 2/5 | total pts-254, gt pts - 135
car #4827: Exported 3/5 | total pts-69, gt pts - 21
car #4828: Exported 5/5 | total pts-6, gt pts - 6
Frame #831/4025
car #4829: Exported 1/7 | total pts-332, gt pts - 135
car #4830: Exported 2/7 | total pts-244, gt pts - 120
car #4831: Exported 3/7 | total pts-160, gt pts - 34
car #4832: Exported 4/7 | total pts-60, gt pts - 38
car #4833: Exported 6/7 | total pts-5, gt pts - 2
car #4834: Exported 7/7 | total pts-7, gt pts - 3
Frame #832/4

car #4961: Exported 5/7 | total pts-15, gt pts - 14
car #4962: Exported 6/7 | total pts-6, gt pts - 3
Frame #873/4025
car #4963: Exported 1/9 | total pts-308, gt pts - 199
car #4964: Exported 2/9 | total pts-107, gt pts - 53
car #4965: Exported 3/9 | total pts-81, gt pts - 30
car #4966: Exported 4/9 | total pts-31, gt pts - 8
car #4967: Exported 5/9 | total pts-43, gt pts - 20
car #4968: Exported 6/9 | total pts-16, gt pts - 14
car #4969: Exported 7/9 | total pts-11, gt pts - 3
car #4970: Exported 8/9 | total pts-5, gt pts - 1
car #4971: Exported 9/9 | total pts-5, gt pts - 4
Frame #874/4025
car #4972: Exported 1/8 | total pts-206, gt pts - 83
car #4973: Exported 2/8 | total pts-209, gt pts - 123
car #4974: Exported 3/8 | total pts-71, gt pts - 37
car #4975: Exported 4/8 | total pts-36, gt pts - 36
car #4976: Exported 5/8 | total pts-14, gt pts - 11
car #4977: Exported 6/8 | total pts-30, gt pts - 7
car #4978: Exported 7/8 | total pts-14, gt pts - 6
car #4979: Exported 8/8 | total pts-

# TODO: Edit to take in detection bbox in the same format as CLOC

e.g. Cascade RCNN KITI BBOX pred for 000000.bin. Each line is a new instance. 1st column is car, column [4:8] is the detection bbox, column 15 is the score (in scale of 1000 for some reason). Not sure what the rest are but that's what CLOC used.
```
Car -1 -1 -10 1133.50 278.19 1225.04 329.51 -1 -1 -1 -1000 -1000 -1000 -10 0.0150 
Car -1 -1 -10 1156.30 225.86 1225.01 262.08 -1 -1 -1 -1000 -1000 -1000 -10 0.0139 
Car -1 -1 -10 1044.50 215.57 1112.86 259.75 -1 -1 -1 -1000 -1000 -1000 -10 0.0021
```